# **I &nbsp;&nbsp;&nbsp; Import Libraries**

In [1]:
import pandas as pd

import re
import emoji
import py_vncorenlp

# **II &nbsp;&nbsp;&nbsp; Data Preprocessing**

## **1 &nbsp;&nbsp;&nbsp; Merge Data**

In [2]:
aivivn_path = '../input/raw/AIVIVN_2019/'
vlsp_path = '../input/raw/VLSP2016_SA/'
uit_path = '../input/raw/UIT_VSFC/'

### **1.1 &nbsp;&nbsp;&nbsp; AIVIVN-2019**

In [3]:
aivivn_train_df = pd.read_csv(aivivn_path + 'train.csv')
aivivn_test_df = pd.read_csv(aivivn_path + 'test.csv')

In [4]:
print(aivivn_train_df.head())
print(aivivn_test_df.head())

             id                                            comment  label
0  train_001908     Shop phục vụ rất kémKo có đạo đức nghề nghiệp       0
1  train_006244  sản phẩm giống hình nhưng áo rộng dài còn quần...      0
2  train_015521   Chất lượng sản phẩm tuyệt vời Chất lượng sản ...      1
3  train_012157   Chất lượng sản phẩm tuyệt vời .Đóng gói sản p...      1
4  train_002251                               Shop tư vấn rất tốt       1
             id                                            comment  label
0  train_009625  Trả lời rất nhiệt tình gói hàng cẩn thận quá t...      1
1  train_004624                   Sản phẩm không đúng như rao bán.      0
2  train_013076  Chất lượng sản phẩm tuyệt vời! Shop phục vụ rấ...      1
3  train_004531                       Bim tốt. Ung hộ shop dai dai      1
4  train_006020  Hộp còn nguyên tem chưa có dấu hiệu bị bóc son...      1


In [5]:
aivivn_train_df = aivivn_train_df.drop(columns='id')
aivivn_test_df = aivivn_test_df.drop(columns='id')

In [6]:
aivivn_train_df = aivivn_train_df.rename(columns={"comment": "Text", "label": "Label"})
aivivn_test_df = aivivn_test_df.rename(columns={"comment": "Text", "label": "Label"})

In [7]:
n = int(0.75 * len(aivivn_train_df))
aivivn_train_df_split = aivivn_train_df.iloc[:n, :]
aivivn_val_df = aivivn_train_df.iloc[n:, :]

In [8]:
print(aivivn_train_df_split.head())
print(aivivn_test_df.head())
print(aivivn_val_df.head())

                                                Text  Label
0     Shop phục vụ rất kémKo có đạo đức nghề nghiệp       0
1  sản phẩm giống hình nhưng áo rộng dài còn quần...      0
2   Chất lượng sản phẩm tuyệt vời Chất lượng sản ...      1
3   Chất lượng sản phẩm tuyệt vời .Đóng gói sản p...      1
4                               Shop tư vấn rất tốt       1
                                                Text  Label
0  Trả lời rất nhiệt tình gói hàng cẩn thận quá t...      1
1                   Sản phẩm không đúng như rao bán.      0
2  Chất lượng sản phẩm tuyệt vời! Shop phục vụ rấ...      1
3                       Bim tốt. Ung hộ shop dai dai      1
4  Hộp còn nguyên tem chưa có dấu hiệu bị bóc son...      1
                                                   Text  Label
9652         Áo dày dặn đẹp fom áo to nhưng hơi ngắn tý      1
9653  Đóng gói sản phẩm rất đẹp và chắc chắn màu đẹp...      1
9654              quần rộng quá..không giống trong hình      0
9655      Không được như ý c

In [9]:
print(f"Train size: {len(aivivn_train_df_split)}, test size: {len(aivivn_test_df)}, val size: {len(aivivn_val_df)}")

Train size: 9652, test size: 3217, val size: 3218


### **1.2 &nbsp;&nbsp;&nbsp; VLSP2016-SA**

In [10]:
vlsp_train_df = pd.read_csv(vlsp_path + 'train.csv')
vlsp_test_df = pd.read_csv(vlsp_path + 'test.csv')
vlsp_val_df = pd.read_csv(vlsp_path + 'val.csv')

In [11]:
print(vlsp_train_df.head())
print(vlsp_test_df.head())
print(vlsp_val_df.head())

                                                text label
0  mình có thể đồng ý Sony khoảng bền này nọ, như...   NEU
1  Hàng này cực nhiều hàng Fake , nếu mua được hà...   POS
2  Hic lỡ tay mua iphone 6s rồi, chắc năm sau mới...   POS
3  Iphone không cho bạn làm 2 việc 1 lúc trên màn...   NEU
4        Không đẹp bằng S6 , nhìn S6 sống động hơn .   NEG
                                                text label
0  Không nên mua chuột cua Logitech, vì dùng nó r...   POS
1  Nói thiệt là mình thì thì chuột nào mình cũng ...   NEG
2                 Xai chuot so nhat bi double click.   NEU
3  Cơ bản là thiết kế ôm chuột chưa đã Như hiện g...   POS
4  Đang dùng mx1. Cũng ngon nhưng chưa đầy năm mà...   NEG
                                                text label
0  Vâng siu phẩm lặn mất tăm sau 1 năm không bán ...   NEG
1                          Sam sung đi đầu công nghệ   POS
2  Chưa được trải nghiệm nên không có nhận xét gì...   POS
3  Vừa mơi đặt hàng em macbook pro 15 inch 2015 x...   P

In [12]:
vlsp_train_df = vlsp_train_df.rename(columns={"text": "Text", "label": "Label"})
vlsp_test_df = vlsp_test_df.rename(columns={"text": "Text", "label": "Label"})
vlsp_val_df = vlsp_val_df.rename(columns={"text": "Text", "label": "Label"})

In [13]:
vlsp_train_df["Label"] = vlsp_train_df["Label"].map({"NEG": 0, "POS": 1, "NEU": 2})
vlsp_test_df["Label"] = vlsp_test_df["Label"].map({"NEG": 0, "POS": 1, "NEU": 2})
vlsp_val_df["Label"] = vlsp_val_df["Label"].map({"NEG": 0, "POS": 1, "NEU": 2})

In [14]:
print(vlsp_train_df.head())
print(vlsp_test_df.head())
print(vlsp_val_df.head())

                                                Text  Label
0  mình có thể đồng ý Sony khoảng bền này nọ, như...      2
1  Hàng này cực nhiều hàng Fake , nếu mua được hà...      1
2  Hic lỡ tay mua iphone 6s rồi, chắc năm sau mới...      1
3  Iphone không cho bạn làm 2 việc 1 lúc trên màn...      2
4        Không đẹp bằng S6 , nhìn S6 sống động hơn .      0
                                                Text  Label
0  Không nên mua chuột cua Logitech, vì dùng nó r...      1
1  Nói thiệt là mình thì thì chuột nào mình cũng ...      0
2                 Xai chuot so nhat bi double click.      2
3  Cơ bản là thiết kế ôm chuột chưa đã Như hiện g...      1
4  Đang dùng mx1. Cũng ngon nhưng chưa đầy năm mà...      0
                                                Text  Label
0  Vâng siu phẩm lặn mất tăm sau 1 năm không bán ...      0
1                          Sam sung đi đầu công nghệ      1
2  Chưa được trải nghiệm nên không có nhận xét gì...      1
3  Vừa mơi đặt hàng em macbook pro 15 in

In [15]:
print(f"Train size: {len(vlsp_train_df)}, test size: {len(vlsp_test_df)}, val size: {len(vlsp_val_df)}")

Train size: 4080, test size: 1050, val size: 1020


### **1.3 &nbsp;&nbsp;&nbsp; UIT-VSFC**

In [16]:
uit_train_df = pd.read_csv(uit_path + 'synthetic_train.csv')
uit_val_df = pd.read_csv(uit_path + 'synthetic_val.csv')

In [17]:
print(uit_train_df.head())
print(uit_val_df.head())

                                            sentence sentiment       topic
0  Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...  negative    facility
1  The university's musical and artistic faciliti...   neutral    facility
2  Phương pháp giảng dạy phù hợp với các đối tượn...   neutral  curriculum
3  Chương trình học giúp tôi trở thành một chuyên...  positive  curriculum
4  Tôi nghĩ rằng chương trình đào tạo có thể có t...   neutral  curriculum
                                            sentence sentiment       topic
0                           Chất lượng vật chất kém.  negative    facility
1  Phần mềm học tập quá khó sử dụng, khiến sinh v...  negative    facility
2  Trường tôi thiếu những tiện ích cơ bản như máy...  negative    facility
3  Cần tạo thêm các hoạt động gắn kết giữa sinh v...   neutral  curriculum
4  Họ rất khoan dung và lượng giác trong quan điể...   neutral      others


In [18]:
uit_train_df = uit_train_df.drop(columns='topic')
uit_val_df = uit_val_df.drop(columns='topic')

In [19]:
uit_train_df = uit_train_df.rename(columns={"sentence": "Text", "sentiment": "Label"})
uit_val_df = uit_val_df.rename(columns={"sentence": "Text", "sentiment": "Label"})

In [20]:
uit_train_df["Label"] = uit_train_df["Label"].map({"negative": 0, "positive": 1, "neutral": 2})
uit_val_df["Label"] = uit_val_df["Label"].map({"negative": 0, "positive": 1, "neutral": 2})

In [21]:
n = int(0.75 * len(uit_train_df))
uit_train_df_split = uit_train_df.iloc[:n, :]
uit_test_df = uit_train_df.iloc[n:, :]

In [22]:
print(uit_train_df_split.head())
print(uit_test_df.head())
print(uit_val_df.head())

                                                Text  Label
0  Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...      0
1  The university's musical and artistic faciliti...      2
2  Phương pháp giảng dạy phù hợp với các đối tượn...      2
3  Chương trình học giúp tôi trở thành một chuyên...      1
4  Tôi nghĩ rằng chương trình đào tạo có thể có t...      2
                                                   Text  Label
6108  Người này là một người chân thành và có sức ản...      2
6109  Các phòng học không có đủ năng lượng để đảm bả...      0
6110          Cô ấy là người rất nhẹ nhàng và tình cảm.      1
6111  Giờ mở cửa của phòng thực hành tại trường quá ...      0
6112  Tôi đã học được nhiều kiến thức mới đáng kinh ...      1
                                                Text  Label
0                           Chất lượng vật chất kém.      0
1  Phần mềm học tập quá khó sử dụng, khiến sinh v...      0
2  Trường tôi thiếu những tiện ích cơ bản như máy...      0
3  Cần tạo thêm các ho

In [23]:
print(f"Train size: {len(uit_train_df_split)}, test size: {len(uit_test_df)}, val size: {len(uit_val_df)}")

Train size: 6108, test size: 2036, val size: 2036


### **1.4 &nbsp;&nbsp;&nbsp; Merge Dataset**

In [24]:
train_df = pd.concat([aivivn_train_df_split, vlsp_train_df, uit_train_df_split], axis=0).reset_index(drop=True)
test_df = pd.concat([aivivn_test_df, vlsp_test_df, uit_test_df], axis=0).reset_index(drop=True)
val_df = pd.concat([aivivn_val_df, vlsp_val_df, uit_val_df], axis=0).reset_index(drop=True)

In [25]:
print(train_df.head())
print(test_df.head())
print(val_df.head())

                                                Text  Label
0     Shop phục vụ rất kémKo có đạo đức nghề nghiệp       0
1  sản phẩm giống hình nhưng áo rộng dài còn quần...      0
2   Chất lượng sản phẩm tuyệt vời Chất lượng sản ...      1
3   Chất lượng sản phẩm tuyệt vời .Đóng gói sản p...      1
4                               Shop tư vấn rất tốt       1
                                                Text  Label
0  Trả lời rất nhiệt tình gói hàng cẩn thận quá t...      1
1                   Sản phẩm không đúng như rao bán.      0
2  Chất lượng sản phẩm tuyệt vời! Shop phục vụ rấ...      1
3                       Bim tốt. Ung hộ shop dai dai      1
4  Hộp còn nguyên tem chưa có dấu hiệu bị bóc son...      1
                                                Text  Label
0         Áo dày dặn đẹp fom áo to nhưng hơi ngắn tý      1
1  Đóng gói sản phẩm rất đẹp và chắc chắn màu đẹp...      1
2              quần rộng quá..không giống trong hình      0
3      Không được như ý cho lắm .cần khô

In [26]:
print(f"Train size: {len(train_df)}, test size: {len(test_df)}, val size: {len(val_df)}")

Train size: 19840, test size: 6303, val size: 6274


## **2 &nbsp;&nbsp;&nbsp; Preprocess**

In [ ]:
class DataPreprocessor:
    def __init__(self):
        self.abbr_df = pd.read_csv("../input/cleaned/abbreviation_dictionary_vn.csv")
        
        # Absolute path to VnCoreNLP
        path_to_VnCoreNLP = "your absolute path to VnCoreNLP folder"
        
        self.rdrword_segmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=path_to_VnCoreNLP)
    
    def standardize_word(self, text):
        # Lower text
        text = text.lower()
        # Remove extra spaces
        text = text.split()
        text = " ".join(text)
        return text.strip()
    
    def remove_special_characters(self, text):
        # Remove emoji
        text = emoji.replace_emoji(text, replace="")
        # Remove punctuation
        pattern = r"[^0-9a-zA-ZÀ-ỹ\s]"
        text = re.sub(pattern, " ", text)
        return text

    def remove_english_text(self, text):
        # Count vietnamese words
        vietnamese_chars = re.findall(r"[À-Ỹà-ỹ]", text)
        # If zero then it's english
        if len(vietnamese_chars) == 0:
            return ""
        
        return text

    def replace_abbreviation(self, text):
        abbr_map = dict(zip(self.abbr_df["abbreviation"], self.abbr_df["meaning"]))
        return " ".join([abbr_map.get(w, w) for w in text.split()])
    
    def preprocess(self, text):
        text = self.standardize_word(text)
        text = self.remove_special_characters(text)
        text = self.remove_english_text(text)
        text = self.replace_abbreviation(text)
        text = " ".join(self.rdrword_segmenter.word_segment(text))
        return text.strip()

In [28]:
data_preprocessor = DataPreprocessor()

In [29]:
train_df["Text"] = train_df["Text"].apply(data_preprocessor.preprocess)
test_df["Text"] = test_df["Text"].apply(data_preprocessor.preprocess)
val_df["Text"] = val_df["Text"].apply(data_preprocessor.preprocess)

In [30]:
# Remove null rows after preprocessing
train_df = train_df[train_df['Text'].notnull() & (train_df['Text'].str.strip() != "")]
test_df = test_df[test_df['Text'].notnull() & (test_df['Text'].str.strip() != "")]
val_df = val_df[val_df['Text'].notnull() & (val_df['Text'].str.strip() != "")]

## **3 &nbsp;&nbsp;&nbsp; Write File**

In [31]:
train_df.to_csv("../data/input/cleaned/train/train.csv", index=False)
test_df.to_csv("../data/input/cleaned/test/test.csv", index=False)
val_df.to_csv("../data/input/cleaned/val/val.csv", index=False)